## Phenotype Selection

Temporary process to select a few phenotypes for testing.

In [1]:
import pandas as pd
import re

In [12]:
dfp = pd.read_csv('gs://rs-ukb/prep-data/main/ukb41430.phesant.csv', sep='\t', nrows=10000, low_memory=False)
dfp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 8117 entries, userId to x110006_0_0
dtypes: float64(6619), int64(13), object(1485)
memory usage: 619.3+ MB


In [13]:
dfp.columns

Index(['userId', 'x21022_0_0', 'x31_0_0', 'x3_0_0', 'x4_0_0', 'x5_0_0',
       'x6_0_0', 'x19_0_0', 'x21_0_0', 'x31_0_0.1',
       ...
       'x104580_0_0', 'x104590_0_0', 'x104670_0_0', 'x104900_0_0',
       'x104910_0_0', 'x104920_0_0', 'x105010_0_0', 'x105030_0_0',
       'x110005_0_0', 'x110006_0_0'],
      dtype='object', length=8117)

In [28]:
def col_name_to_field_id(col: str) -> int:
    field_id = re.search(r'x(\d+)_\d+_\d+', col)
    if not field_id:
        raise ValueError(f'Failed to find field id in column {col}')
    return int(field_id.group(1))

def field_id_to_col_name(field_id: str) -> str:
    return f'x{field_id}_0_0'

### Phenotype EDA

In [29]:
dfd = pd.read_csv('gs://rs-ukb/pipe-data/external/ukb_meta/data_dictionary_showcase.csv')
field_id_to_name = dfd.set_index('FieldID')['Field'].to_dict()
fields = dfp.columns[1:].map(col_name_to_field_id).map(lambda c: (c, field_id_to_name[c]))
fields = pd.Series(dict(fields.values))

In [41]:
#dfd.pipe(lambda df: df[df['Field'].str.lower().str.contains('vascular disorders of intestine')])

In [35]:
dfd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7460 entries, 0 to 7459
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Path          7460 non-null   object 
 1   Category      7460 non-null   int64  
 2   FieldID       7460 non-null   int64  
 3   Field         7460 non-null   object 
 4   Participants  7460 non-null   int64  
 5   Items         7460 non-null   int64  
 6   Stability     7460 non-null   object 
 7   ValueType     7460 non-null   object 
 8   Units         3262 non-null   object 
 9   ItemType      7460 non-null   object 
 10  Strata        7460 non-null   object 
 11  Sexed         7460 non-null   object 
 12  Instances     7460 non-null   int64  
 13  Array         7460 non-null   int64  
 14  Coding        3945 non-null   float64
 15  Notes         7434 non-null   object 
 16  Link          7460 non-null   object 
dtypes: float64(1), int64(6), object(10)
memory usage: 990.9+ KB


In [30]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)
fields[fields.str.lower().str.contains('diagnosed')].head(500)

2443                                                                      Diabetes diagnosed by doctor
2453                                                                        Cancer diagnosed by doctor
2473                                    Other serious medical condition/disability diagnosed by doctor
2966                                                                 Age high blood pressure diagnosed
2976                                                                            Age diabetes diagnosed
3627                                                                              Age angina diagnosed
3761                                                       Age hay fever, rhinitis or eczema diagnosed
3786                                                                              Age asthma diagnosed
3894                                                                        Age heart attack diagnosed
3992                                                        Age emphysema

In [16]:
select_field_ids = [
    4689,
    4700,
    3786,
    2976,
    2443,
    5923,
    21024,
    22128,
    22129,
    22130,
    22131,
    22133,
    22157,
    22158,
    22141,
]
fields.loc[select_field_ids]

4689                                            Age glaucoma diagnosed
4700                                            Age cataract diagnosed
3786                                              Age asthma diagnosed
2976                                            Age diabetes diagnosed
2443                                      Diabetes diagnosed by doctor
5923                                Age macular degeneration diagnosed
21024                                          Ever diagnosed with IBS
22128                                       Doctor diagnosed emphysema
22129                              Doctor diagnosed chronic bronchitis
22130    Doctor diagnosed COPD (chronic obstructive pulmonary disease)
22131                                 Doctor diagnosed cystic fibrosis
22133                                     Doctor diagnosed sarcoidosis
22157                             Age tuberculosis diagnosed by doctor
22158                                Age silicosis diagnosed by doctor
22141 

In [34]:
dfp[fields.loc[select_field_ids].index.map(field_id_to_col_name)].describe()

,x4689_0_0,x4700_0_0,x3786_0_0,x2976_0_0,x2443_0_0,x5923_0_0,x21024_0_0,x22128_0_0,x22129_0_0,x22130_0_0,x22131_0_0,x22133_0_0,x22157_0_0,x22158_0_0,x22141_0_0
count,70.000000,155.000000,1149.000000,504.000000,9981.000000,33.000000,3447.000000,2417.000000,2417.000000,2417.000000,2417.0,2417.000000,10.000000,0.0,2417.0
mean,53.271429,55.464516,27.210618,50.192460,0.048492,57.878788,-3.893240,0.002896,0.011585,0.013653,0.0,0.002896,10.200000,NaN,0.0
std,14.447227,14.472220,20.390804,14.669611,0.239516,5.935895,28.327221,0.053749,0.107029,0.116071,0.0,0.053749,7.699928,NaN,0.0
min,-1.000000,-1.000000,-1.000000,-1.000000,-3.000000,44.000000,-818.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,NaN,0.0
25%,49.000000,53.500000,7.000000,45.750000,0.000000,55.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,4.500000,NaN,0.0
50%,57.500000,60.000000,28.000000,55.000000,0.000000,59.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,9.500000,NaN,0.0
75%,63.000000,63.500000,45.000000,60.000000,0.000000,63.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,13.750000,NaN,0.0
max,69.000000,69.000000,68.000000,70.000000,1.000000,67.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,27.000000,NaN,0.0


### Phenotype Selection

In [45]:
# Load sample ids taken from NL result sheet
dfsid = pd.read_csv('gs://rs-ukb/pipe-data/external/nealelab_v3_20180731/extract/ukb_field_ids_from_showcase_link.csv')
dfsid.head()

,field_id
0,46
1,47
2,48
3,49
4,50


In [53]:
fields

21022                                    Age at recruitment
31                                                      Sex
3                                 Verbal interview duration
4                                       Biometrics duration
5                                Sample collection duration
                                ...                        
104920             Time spent doing light physical activity
105010                    When diet questionnaire completed
105030                      When diet questionnaire started
110005    Invitation to physical activity study, acceptance
110006     Invitation to physical activity study, date sent
Length: 2383, dtype: object

In [60]:
# Summary for phenotypes in intersection of PHESANT sample and NL results
col_names = {
    field_id_to_col_name(field_id): (field_id, name)
    for field_id, name in fields.iteritems()
}
dfp_desc = dfp.filter(items=dfsid['field_id'].map(field_id_to_col_name).values).rename(columns=col_names)
dfp_desc.columns = pd.MultiIndex.from_tuples(dfp_desc.columns)
dfp_desc = dfp_desc.describe().T
dfp_desc

,,count,mean,std,min,25%,50%,75%,max
46,Hand grip strength (left),9937.0,29.680688,11.326279,0.0,21.0,28.0,38.0,83.0
47,Hand grip strength (right),9936.0,31.862419,11.273410,0.0,24.0,30.0,40.0,74.0
48,Waist circumference,9961.0,90.389268,13.516509,56.0,80.0,90.0,99.0,182.0
49,Hip circumference,9961.0,103.389449,9.168426,76.0,97.0,102.0,108.0,179.0
50,Standing height,9952.0,168.496322,9.181197,139.0,162.0,168.0,175.0,204.0
...,...,...,...,...,...,...,...,...,...
104530,Orange intake,154.0,64.870130,176.171548,1.0,1.0,1.0,1.0,555.0
104670,Vitamin supplement user,1370.0,0.397810,0.489625,0.0,0.0,0.0,1.0,1.0
104900,Time spent doing vigorous physical activity,1370.0,394.140146,889.766252,0.0,0.0,0.0,12.0,3060.0
104910,Time spent doing moderate physical activity,1370.0,690.007299,1053.274649,0.0,10.0,12.0,1030.0,3060.0


In [61]:
# How can we get summary stats for any of the above?
# - OTG may only have those from the ICD phenotypes

### Open Targets EFO Mappings to UKB

In [63]:
dfefo = pd.read_csv('gs://rs-ukb/pipe-data/external/ukb_meta/efo_mapping.csv', sep='\t')
dfefo.head()

,ZOOMA QUERY,MAPPED_TERM_LABEL,MAPPED_TERM_URI,MAPPING_TYPE,ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE,COMMENTS/TICKET,AI
0,Vascular disorders of intestine,vascular disease,"EFO_0004264, EFO_0009431",Broad,K55,DONE,NaN
1,Gonarthrosis,osteoarthritis || knee,EFO_0004616,Broad,M17,DONE,NaN
2,Psoriatic and enteropathic arthropathies,psoriatic arthritis,EFO_0003778,? Broad,M07,DONE,NaN
3,Pain associated with micturition,dysuria,EFO_0003901,? Broad,R30,DONE,NaN
4,Other mood,mood disorder,EFO_0004247,? Broad,F38,DONE,NaN


In [69]:
print(dfefo['ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE'].tolist())

['K55', 'M17', 'M07', 'R30', 'F38', 'O60', 'S54', 'S64', 'K29', 'Z58', 'O10', 'O11', 'S65', 'T00', 'T01', 'T06', 'T70', 'T91', 'T94', 'T98', 'T09', 'Z85', 'M05', 'L91', 'B88', 'K74', 'M15', 'Z34', 'N26', 'O23', 'I69', 'I24', 'I51', 'M82', 'F13', 'F16', 'F18', 'S32', 'G60', 'M90', 'G25', 'Q68', 'Z22', 'O42', 'A60', 'M03', 'A51', 'T97', 'E02', 'H45', 'M30', 'G09', 'A16', 'H13', 'Q65', 'P59', 'N35', 'M12', 'H90', 'S49', 'T67', 'H74', 'A81', 'N81', 'K07', 'Q37', 'Q30', 'H52', 'Q44', 'O65', 'R84', 'Q33', 'Q27', 'I83', 'I86', 'L04', 'R82', 'K46', 'B19', 'Q45', 'Q28', 'Q82', 'J63', 'L84', 'H06', 'G30', 'I38', 'Z84', 'O16', 'I64', 'I60', 'B24', 'E34', 'K76', 'G71', 'F84', 'I67', 'K82', 'R52', 'T08', 'B16', 'R00', 'M41', 'Q24', 'K00', 'I61', 'H33', 'A17', 'B15', 'B76', 'N97', 'G47', 'E51', 'B30', 'K70', 'M42', 'K11', 'F64', 'J46', 'Q61', 'N84', 'F34', 'H43', 'E28', 'D68', 'L92', 'O21', 'E22', 'E63', 'I09', 'B83', 'A89', 'K57', 'O45', 'R33', 'R32', 'R80', 'Q35', 'G91', 'H53', 'F03', 'R31', 'K81'

In [80]:
df_map_p = pd.DataFrame([
    dict(
        map_code=code,
        col_name=field_id_to_col_name(code.split('_')[0]),
        in_phenotypes=field_id_to_col_name(code.split('_')[0]) in dfp
    )
    for code in dfefo['ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE'].dropna().tolist()
])
df_map_p.head()

,map_code,col_name,in_phenotypes
0,K55,xK55_0_0,False
1,M17,xM17_0_0,False
2,M07,xM07_0_0,False
3,R30,xR30_0_0,False
4,F38,xF38_0_0,False


In [81]:
df_map_p['in_phenotypes'].value_counts()

False    1021
True      543
Name: in_phenotypes, dtype: int64

In [82]:
pd.set_option('display.max_rows', None)
df_map_p.pipe(lambda df: df[df['in_phenotypes']])\
    .assign(field_id=lambda df: df['col_name'].apply(col_name_to_field_id))\
    .assign(field_name=lambda df: df['field_id'].map(fields))\
    .drop_duplicates()

,map_code,col_name,in_phenotypes,field_id,field_name
564,48,x48_0_0,True,48,Waist circumference
565,49,x49_0_0,True,49,Hip circumference
566,50,x50_0_0,True,50,Standing height
567,102,x102_0_0,True,102,"Pulse rate, automated reading"
568,1160,x1160_0_0,True,1160,Sleep duration
569,1180,x1180_0_0,True,1180,Morning/evening person (chronotype)
570,1200,x1200_0_0,True,1200,Sleeplessness / insomnia
571,1210,x1210_0_0,True,1210,Snoring
572,1558,x1558_0_0,True,1558,Alcohol intake frequency.
573,1920,x1920_0_0,True,1920,Mood swings
